In [1]:
from architectures_v1 import *
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, RNN
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sklearn.metrics import f1_score, precision_score, recall_score

import os
import numpy as np
import tensorflow as tf
import pandas as pd
import time

## Loading Training/Testing/Validation Data

In [2]:
dataset = "ChlorineConcentration"

x_train_load = np.load(os.path.abspath(os.path.join('99_data', dataset,'x_train.npz')))
x_test_load = np.load(os.path.abspath(os.path.join('99_data', dataset,'x_test.npz')))

x_train = np.reshape(x_train_load['arr_0'], [x_train_load['arr_0'].shape[0], x_train_load['arr_0'].shape[1], 1])
x_test = np.reshape(x_test_load['arr_0'], [x_test_load['arr_0'].shape[0], x_test_load['arr_0'].shape[1], 1])

x_all = np.concatenate((x_train, x_test), axis = 0)

n_instances = x_all.shape[0]

y_train_load = np.load(os.path.abspath(os.path.join('99_data', dataset,'y_train.npz')))
y_test_load = np.load(os.path.abspath(os.path.join('99_data', dataset,'y_test.npz')))

y_train = y_train_load['arr_0']
y_test = y_test_load['arr_0']

y_all = np.concatenate((y_train, y_test), axis = 0)
y_all = np.asarray(y_all, dtype = np.uint8)

x_valid = x_all[:int(0.1*n_instances), :, :]
y_valid = y_all[:int(0.1*n_instances)]

x_all = x_all[int(0.1*n_instances):, :, :]
y_all = y_all[int(0.1*n_instances):]

print(f"x_all: {x_all.shape} - y_all: {y_all.shape}")
print(f"x_valid: {x_valid.shape} - y_valid: {y_valid.shape}")

x_all: (3877, 166, 1) - y_all: (3877,)
x_valid: (430, 166, 1) - y_valid: (430,)


In [3]:
seed = 42
tf.random.set_random_seed(seed)
np.random.seed(seed)

length = x_all.shape[1]
n_features = x_all.shape[-1]

model = Sequential()

model.add(RNN(NeuromodulatedBistableRecurrentCellLayer(output_dim = 212),
              input_shape = (length, n_features), 
              return_sequences = False))
model.add(Dense(to_categorical(y_all).shape[-1], activation = "softmax"))

model.compile(optimizer = "adam", loss = "categorical_crossentropy", metrics = ["accuracy"])
model.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rnn (RNN)                    (None, 212)               90948     
_________________________________________________________________
dense (Dense)                (None, 3)                 639       
Total params: 91,587
Trainable params: 91,587
Non-trainable params: 0
_________________________________________________________________


In [4]:
if not os.path.exists(os.path.abspath(os.path.join('models', dataset))):
    os.mkdir(os.path.abspath(os.path.join('models', dataset)))

kf = KFold(n_splits = 5)
sdk = StratifiedKFold(n_splits = 5, random_state = 42, shuffle = True)
    
n_fold = 1

acc_per_fold = []
epoch_per_fold = []
loss_per_fold = []
rec_per_fold = []
prec_per_fold = []
f1_per_fold = []

for train, test in sdk.split(x_all, y_all):
    
    file_path = os.path.abspath(os.path.join('models', dataset, f'{dataset}-nBRC_{n_fold}.hdf5'))

    callbacks = [
        ModelCheckpoint(filepath=file_path, monitor='val_loss', save_best_only = True, mode = 'min', verbose = 1), 
        EarlyStopping(monitor = 'val_loss', patience = 50, mode = 'min')]

    t = time.time()
    
    print(f"Fold: {n_fold}\n")
    result = model.fit(x_all[train], 
                       to_categorical(y_all[train]),
                       epochs = 2000, 
                       batch_size = 100, 
                       validation_data = (x_valid, to_categorical(y_valid)), 
                       callbacks = callbacks)

    print(f"Training time: {time.time() - t} s")

    df_results = pd.DataFrame(result.history)
    df_results.to_csv(os.path.abspath(os.path.join('models', dataset, f'nBRC_results_{n_fold}.csv')))
    
    model.load_weights(file_path)
    scores = model.evaluate(x_all[test], to_categorical(y_all[test]))
    loss_per_fold.append(scores[0])
    acc_per_fold.append(scores[1])
    epoch_per_fold.append(np.argmin(result.history['val_loss']))
    
    # Computing predictions
    y_pred_test = np.argmax(model.predict(x_all[test]), axis = 1)
    
    f1_per_fold.append(f1_score(y_all[test], y_pred_test, average = 'macro'))
    rec_per_fold.append(recall_score(y_all[test], y_pred_test, average = 'macro'))
    prec_per_fold.append(precision_score(y_all[test], y_pred_test, average = 'macro'))
    
    n_fold += 1
    
df_scores = pd.DataFrame(columns = ['F1', 'Loss', 'Accuracy', 'Precision', 'Recall'])
df_scores['F1'] = f1_per_fold
df_scores['Loss'] = loss_per_fold
df_scores['Accuracy'] = acc_per_fold
df_scores['Precision'] = prec_per_fold
df_scores['Recall'] = rec_per_fold

df_scores.to_csv(os.path.abspath(os.path.join('models', dataset, f'nBRC_results_k-Folds.csv')))

Fold: 1

Train on 3101 samples, validate on 430 samples
Epoch 1/2000
3100/3101 [============================>.] - ETA: 0s - loss: 1.0965 - acc: 0.4542
Epoch 00001: val_loss improved from inf to 1.00360, saving model to C:\Users\Sergio\Documents\GitHub_Repositories\01_Maintained\NeuralODE\models\ChlorineConcentration\ChlorineConcentration-nBRC_1.hdf5
3101/3101 [==============================] - 4s 1ms/sample - loss: 1.0967 - acc: 0.4540 - val_loss: 1.0036 - val_acc: 0.5651
Epoch 2/2000
3100/3101 [============================>.] - ETA: 0s - loss: 1.0351 - acc: 0.5123
Epoch 00002: val_loss improved from 1.00360 to 0.99877, saving model to C:\Users\Sergio\Documents\GitHub_Repositories\01_Maintained\NeuralODE\models\ChlorineConcentration\ChlorineConcentration-nBRC_1.hdf5
3101/3101 [==============================] - 4s 1ms/sample - loss: 1.0352 - acc: 0.5121 - val_loss: 0.9988 - val_acc: 0.5488
Epoch 3/2000
3100/3101 [============================>.] - ETA: 0s - loss: 1.0284 - acc: 0.5268
Epo

c:\users\sergio\.conda\envs\sd_dev_tf_1\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


3100/3101 [============================>.] - ETA: 0s - loss: 0.9967 - acc: 0.5481
Epoch 00001: val_loss improved from inf to 0.98660, saving model to C:\Users\Sergio\Documents\GitHub_Repositories\01_Maintained\NeuralODE\models\ChlorineConcentration\ChlorineConcentration-nBRC_2.hdf5
3101/3101 [==============================] - 3s 1ms/sample - loss: 0.9966 - acc: 0.5482 - val_loss: 0.9866 - val_acc: 0.5721
Epoch 2/2000
3100/3101 [============================>.] - ETA: 0s - loss: 1.0023 - acc: 0.5455
Epoch 00002: val_loss did not improve from 0.98660
3101/3101 [==============================] - 3s 1ms/sample - loss: 1.0022 - acc: 0.5456 - val_loss: 0.9866 - val_acc: 0.5721
Epoch 3/2000
3100/3101 [============================>.] - ETA: 0s - loss: 0.9969 - acc: 0.5487- ETA: 0s - loss: 0.9986 - acc: 0.54
Epoch 00003: val_loss improved from 0.98660 to 0.97976, saving model to C:\Users\Sergio\Documents\GitHub_Repositories\01_Maintained\NeuralODE\models\ChlorineConcentration\ChlorineConcentrati